<font size="10"> Big Contest 2019 </font>

-------------------------------
<br />

# 전처리

모든 팀원이 쉽게 이해할 수 있는 코드를 작성하여 효율적인 프로젝트 진행을 목표로 합니다.
<br />
<br />
**변수 생성 기준**

- 예측 정확도 상승
- 편의

**객체(변수, 함수) 네이밍 규칙**

- 고유명사가 아닌 줄임말은 명확하도록 풀어 적는다. e.g.) STT -> sched_time
- 줄임말 기준: 
    - 줄임말을 썻을 때 이해하기 쉬운 경우
    - 기존 이름이 너무 장황
    - 일반적으로 많이 쓰는 줄임말을 사용. 
    - e.g.) count -> cnt, average -> avg, schedule -> sched 
- 동사를 앞에 적는다. 동사\_명사 e.g.) sum_flight, avg_flight, cnt_flight
- 자주 사용되는 단어일수록 앞에 위치 e.g.) sched_time, sched_datetime
- 더미변수(binary 변수)는 is_이름 e.g.) is_delay, is_holiday
- 생소하거나 어려운 단어라면 길더라도 그냥 쓰자.
- "."을 사용하지말고 "\_"로 통일하자. "."을 사용하게 되면 메소드와 혼돈을 일으킴.
- GPA와 같은 이름을 제외하면 소문자를 사용하자.
- 영어 이름을 사용하자.

**가독성**

- 주석을 달아주자.
- 다음은 선택사항입니다.
    - 코드 실행 단축을 위한 벡터화: 간단히 설명하자면 for루프, apply 대신 판다스, 사이킨런 내장함수를 사용하면 된다.
    - 반복된 코드로 스크립트가 길어지는 것을 방지하기 위해 함수화하자. 
    - 메모리를 위해 가끔식 객체 삭제가 필요할 수도 있다.
- 파이썬 스타일 가이드 PEP8을 지키자. 적어도 망치를 눌러주자.

<br />

<font size="4"> 모든 데이터셋에 대해서 다음과 같은 변수를 추가했습니다. </font>

- 날짜(date): yyyy-mm-dd
- 날짜시간(datetime): yyyy-mm-dd hh:mm:ss
- is_로 시작하는 binary 변수의 값은 True, False로 정하였습니다. 여러 연산을 수행할 때 훨씬 편하기 때문입니다. 
 e.g.) `sum(df_afsnt['is_delay'])`

EDA를 기준으로 작성하였으며 모델링용 데이터셋은 나중에 다시 작성하겠습니다.

In [12]:
import pandas as pd

df_afsnt = pd.read_csv("AFSNT.csv", encoding="CP949")
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))
df_afsnt["is_arrive"] = np.where(df_afsnt["is_arrive"] == "A", "Y", "N")
df_afsnt[[col for col in df_afsnt if col.startswith('is_')]] = df_afsnt[[col for col in df_afsnt if col.startswith('is_')]].apply(lambda x: x == "Y") # is_가 많아졌을 때 효과적인 코딩임

# 공항 테이블
df_afsnt['origin'] = df_afsnt['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
df_afsnt['dest'] = df_afsnt['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})

# 항공사 테이블 
df_afsnt['airline'] = df_afsnt['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

df_afsnt.head()
# df_afsnt.to_csv("df_afsnt.csv", index = False, encoding='CP949')

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,sched_time,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay
0,2017,1,1,일,제주,청주,대한항공,J1955,SEw3NzE4,False,...,10:05:00,10:32:00,False,NaN,False,NaN,2017-01-01,2017-01-01 10:05:00,2017-01-01 10:32:00,27
1,2017,1,1,일,제주,청주,대한항공,J1954,SEw3NzE4,True,...,09:30:00,09:31:00,False,NaN,False,NaN,2017-01-01,2017-01-01 09:30:00,2017-01-01 09:31:00,1
2,2017,1,1,일,제주,청주,대한항공,J1956,SEw3NzE4,True,...,12:45:00,13:03:00,False,NaN,False,NaN,2017-01-01,2017-01-01 12:45:00,2017-01-01 13:03:00,18
3,2017,1,1,일,제주,청주,대한항공,J1957,SEw3NzE4,False,...,13:25:00,14:09:00,True,C02,False,NaN,2017-01-01,2017-01-01 13:25:00,2017-01-01 14:09:00,44
4,2017,1,1,일,제주,청주,대한항공,J1958,SEw3NzE4,True,...,16:10:00,16:31:00,False,NaN,False,NaN,2017-01-01,2017-01-01 16:10:00,2017-01-01 16:31:00,21


In [23]:
import pandas as pd

# df_afsnt = pd.read_csv("df_afsnt.csv", encoding = "cp949")
temp = df_afsnt.groupby(["origin_dest", "is_weekend"]).agg({"is_cancel": ["mean", "sum"]}).reset_index()
temp.to_csv("주말공항지연비율.csv", index = False, encoding = "cp949")

## 시간변수

<br />
<br />

**year feature** 
- 사드: 제주도  
- 일본: 제주도
- 기상악화: 태풍, 폭우 기타 등등

<br />

**month feature**
- month_sin: $sin(\frac{2\pi \ \times \ month}{12})$ month_cos: $cos(\frac{2\pi \ \times \ month}{12})$ 
- 성수기 
- 기타 

**day feature**

- is_weekend: 주말
- is_holiday: 공휴일
- is_goldtime: 황금연휴 (3일 또는 4일 이상 연속적으로 공휴일)
- goldtime_distance: 황금연휴로부터 얼마나 떨어졌는지(19년 9월15일까지가 추석연휴임) 

**hour, minute feature**
- time_sin: $60 sin(\frac{2\pi \ \times \ hour}{24}) + sin(\frac{2\pi \ \times \ minute}{60})$ time_cos: $60 cos(\frac{2\pi \ \times \ hour}{24}) + cos(\frac{2\pi \ \times \ minute}{60})$ 
- is_morning, is_launch, is_dinner: 나누는 기준?

In [4]:
import datetime as dt

df_afsnt['sched_datetime'] = pd.to_datetime(df_afsnt['sched_datetime'])

# 월
df_afsnt['month_sin'] = np.sin(2*df_afsnt['sched_datetime'].dt.month*np.pi / 12 )
df_afsnt['month_cos'] = np.cos(2*df_afsnt['sched_datetime'].dt.month*np.pi / 12 )

# 요일
df_afsnt['sched_datetime'] = pd.to_datetime(df_afsnt['sched_datetime'])
df_afsnt['is_weekend'] = np.where((df_afsnt['sched_datetime'].dt.weekday == 6) | (df_afsnt['sched_datetime'].dt.weekday == 0), True, False)

# 시간
df_afsnt['time_sin'] = 60 * np.sin(2*np.pi*df_afsnt['sched_datetime'].dt.hour / 24) + \
                        np.sin(2*df_afsnt['sched_datetime'].dt.minute*np.pi / 60)
df_afsnt['time_cos'] = 60 * np.cos(2*df_afsnt['sched_datetime'].dt.hour*np.pi / 24) + \
                        np.cos(2*np.pi*df_afsnt['sched_datetime'].dt.minute / 60)

## 공공데이터 

### 노선별 소요시간 및 거리 정보 + 해발고도



In [13]:
df_flight_airtime = pd.read_csv("df_flight_airtime.csv")[:43]

# 변수 이름 변경
df_flight_airtime = df_flight_airtime.rename(columns = {
                                    'ARP': 'origin', 
                                    'ODP': 'dest', 
                                    'KM': 'distance',
                                    'TIME': 'time'}
                                    )

# 변수 값 변경
df_flight_airtime['cityKor'] = \
    df_flight_airtime['cityKor'].replace({
    '서울/김포': '김포',
    '부산/김해': '김해',
    '횡성/원주': '원주'
})

df_flight_airtime['origin'] = \
    df_flight_airtime['origin'].replace({
    'CJJ': '청주',
    'CJU': '제주',
    'GMP': '김포',
    'HIN': '사천',
    'ICN': '인천',
    'KPO': '포항',
    'KUV': '군산',
    'KWJ': '광주',
    'MWX': '무안',
    'PUS': '부산',
    'RSU': '여수',
    'TAE': '대구',
    'USN': '울산',
    'WJU': '원주',
    'YNY': '양양'
})
df_flight_airtime['dest'] = \
    df_flight_airtime['dest'].replace({
    'CJJ': '청주',
    'CJU': '제주',
    'GMP': '김포',
    'HIN': '사천',
    'ICN': '인천',
    'KPO': '포항',
    'KUV': '군산',
    'KWJ': '광주',
    'MWX': '무안',
    'PUS': '부산',
    'RSU': '여수',
    'TAE': '대구',
    'USN': '울산',
    'WJU': '원주',
    'YNY': '양양'
})
# 불필요 변수 제거
df_flight_airtime.drop(['cityCode', 'cityEng', 'cityKor'], axis = 1, inplace = True)

# foreign key 생성
df_flight_airtime['origin_dest'] = df_flight_airtime['origin'] + '_' + df_flight_airtime['dest']
df_flight_airtime.drop(['origin', 'dest'], axis = 1, inplace = True)

df_afsnt['origin_dest'] = df_afsnt['origin'] + '_' + df_afsnt['dest']

# df_afsnt에 df_flight_airtime을 Left_join
df_afsnt = pd.merge(df_afsnt, df_flight_airtime, how = "left")

In [14]:
# 해발 고도 
df_altitude = pd.DataFrame(pd.Series({
    '김포': 18,
    '김해': 4,
    '제주': 36,
    '대구': 35.4,
    '울산': 12.9,
    '청주': 58,
    '양양': 73.5,
    '무안': 23.7,
    '광주': 12.8,
    '여수': 20,
    '사천': 7.6,
    '포항': 21.3,
    '군산': 8.8,
    '원주': 100,
    '인천': 7
})).reset_index().rename(columns = {'index': 'airport', 0: 'alt'})

df_afsnt = pd.merge(df_afsnt, df_altitude, how = "left", left_on = 'origin', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_afsnt = pd.merge(df_afsnt, df_altitude, how = "left", left_on = 'dest', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_afsnt.drop(['airport_origin', 'airport_dest'], axis = 1, inplace = True)

### 부지, 활주로, 터미널 정보

인천공항의 정보를 찾아야한다.
현재 인청공항의 연간처리능력을 찾지 못한 상태여서 매핑은 보류

In [16]:
df_airstrip = pd.read_csv("공공데이터\공항별부지및활주로정보.csv", encoding = 'cp949')
pd.merge(df_afsnt, df_airstrip, how = "left", left_on = "origin", right_on = "공항명")
# df_afsnt = pd.merge(df_afsnt, df_airstrip, how = "left", left_on = "origin", right_on = "공항명", suffixes = ('_origin', '_dest'))
# df_afsnt = pd.merge(df_afsnt, df_airstrip, how = "left", left_on = "dest", right_on = "공항명", suffixes = ('_origin', '_dest'))
# df_afsnt

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,origin_dest,distance,time,alt_origin,alt_dest,공항명,lot_area,main_airstrip_standard,sub_airstrip_standard,processing_capacity
0,2017,1,1,일,제주,청주,대한항공,J1955,SEw3NzE4,False,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
1,2017,1,1,일,제주,청주,대한항공,J1954,SEw3NzE4,True,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
2,2017,1,1,일,제주,청주,대한항공,J1956,SEw3NzE4,True,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
3,2017,1,1,일,제주,청주,대한항공,J1957,SEw3NzE4,False,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
4,2017,1,1,일,제주,청주,대한항공,J1958,SEw3NzE4,True,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
5,2017,1,1,일,제주,청주,대한항공,J1959,SEw3NTk5,False,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
6,2017,1,1,일,제주,청주,대한항공,J1960,SEw3NTk5,True,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
7,2017,1,1,일,제주,청주,대한항공,J1961,SEw3NTk5,False,...,제주_청주,367.0,42.0,36.0,58.0,NaN,NaN,NaN,NaN,NaN
8,2017,1,1,일,김해,제주,대한항공,J1015,SEw3NzA2,True,...,김해_제주,NaN,NaN,4.0,36.0,NaN,NaN,NaN,NaN,NaN
9,2017,1,1,일,김포,제주,대한항공,J1242,SEw3NzA2,False,...,김포_제주,449.0,48.0,18.0,36.0,NaN,NaN,NaN,NaN,NaN


### 공항별여객및화물터미널정보

인천공항의 17년, 18년, 19년 여객터미널 면적(㎡), 여객터미널 수용능력(만명/년), 여객터미널, 화물터미널 면적(㎡), 화물터미널 처리능력찾기.

### 공항별체크인카운터운영현황정보

인천공항의 카운터, 컨베이어벨트, 발권데스크, 키오스크 찾아야하고 19년 데이터는 항공사를 찾아야하는데.. 보류

## 항공사 데이터

1. 공공데이터는 대부분이 공항에 대한 정보였기 때문에 구글에서 항공사 데이터를 추가하자.
<br />
<br />
2. feature engineering

    - mean encoding: [mean encoding의 필요성](https://towardsdatascience.com/why-you-should-try-mean-encoding-17057262cd0)



## Test Set

In [8]:
df_test = pd.read_csv("AFSNT_DLY.csv", encoding="CP949")
df_test["STT"] = pd.to_datetime(df_test["STT"], format = "%H:%M").dt.time

df_test = df_test.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", 
    "DLY":"is_delay", "DLY_RATE":"prob_delay"
})

df_test['origin'] = df_test['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
df_test['dest'] = df_test['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})


# # 항공사 테이블 
df_test['airline'] = df_test['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})
df_test["date"] = pd.to_datetime(df_test[["year", "month", "day"]])
df_test["sched_datetime"] = pd.to_datetime(df_test['date'].astype(str) + " " + df_test["sched_time"].astype(str))

df_test.head()
# df_test.to_csv("df_test.csv", index = False, encoding='CP949')

In [30]:
df_test['origin_dest'] = df_test['origin'] + '_' + df_test['dest']
# df_afsnt에 Left_join
df_test = pd.merge(df_test, df_flight_airtime, how = "left")

In [ ]:
df_test = pd.merge(df_test, df_altitude, how = "left", left_on = 'origin', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_test = pd.merge(df_test, df_altitude, how = "left", left_on = 'dest', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_test.drop(['airport_origin', 'airport_dest'], axis = 1, inplace = True)

## SFSNT.csv

In [9]:
df_sched = pd.read_csv("SFSNT.csv", dtype={"FSD":str, "FED":str}, encoding="CP949")
df_sched = df_sched.drop("SSC", 1)
# print(df_sched.apply(lambda x: np.sum(pd.isnull(x)))) # 결측치 존재 
df_sched.loc[:, ["STA", "STD"]] = df_sched[["STA", "STD"]].fillna("00:00")
df_sched["STA"] = pd.to_datetime(df_sched["STA"], format = "%H:%M").dt.time.astype(str) # 맨앞에 0을 만들기 위해
df_sched["STD"] = pd.to_datetime(df_sched["STD"], format = "%H:%M").dt.time.astype(str)

df_sched = df_sched.rename(columns={ 
    "ORG":"origin", "DES":"dest", 
    "FLO":"airline", "FLT":"flight",  
    "STD":"dep_time", "STA":"arr_time",  # dep: depart, arr: arrive 
    "FSD":"start_date", "FED":"end_date", 
    "IRR":"is_regular" 
})



df_sched["start_date"] = df_sched['start_date'].str[:4] + "-" + df_sched['start_date'].str[4:6] + "-" + df_sched['start_date'].str[6:]
df_sched["end_date"] = df_sched['end_date'].str[:4] + "-" + df_sched['end_date'].str[4:6] + "-" + df_sched['end_date'].str[6:]
df_sched["start_dep_datetime"] = pd.to_datetime(df_sched["start_date"].astype(str) + " " + df_sched["dep_time"].astype(str))
df_sched["start_arr_datetime"] = pd.to_datetime(df_sched["start_date"].astype(str) + " " + df_sched["arr_time"].astype(str))
df_sched["end_dep_datetime"] = pd.to_datetime(df_sched["end_date"].astype(str) + " " + df_sched["dep_time"].astype(str))
df_sched["end_arr_datetime"] = pd.to_datetime(df_sched["end_date"].astype(str) + " " + df_sched["arr_time"].astype(str))

# 공항 테이블
df_sched['origin'] = df_sched['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
df_sched['dest'] = df_sched['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})


# # 항공사 테이블 
df_sched['airline'] = df_sched['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})
df_sched.head()
# df_sched.to_csv("df_sched.csv", index = False, encoding='CP949')

In [31]:
df_sched['origin_dest'] = df_sched['origin'] + '_' + df_sched['dest']
# df_afsnt에 Left_join
df_sched = pd.merge(df_sched, df_flight_airtime, how = "left")

In [53]:
df_sched = pd.merge(df_sched, df_altitude, how = "left", left_on = 'origin', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_sched = pd.merge(df_sched, df_altitude, how = "left", left_on = 'dest', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_sched.drop(['airport_origin', 'airport_dest'], axis = 1, inplace = True)

# SMOTE

기존에 존재하지 않는 데이터셋을 만들어내서 클래스 불균형 문제를 해결하는 방법.

훈련셋과 테스트셋의 기간 차이가 77일이나 나기 때문에 개인적으로는 회의적이다.

# 모델링

## Model Evaluation

1. [Nested Cross Validation](https://woolulu.tistory.com/76): 글쓴이가 미래의 데이터에 적용하기 위한 예측 모델을 찾는데는 거의 사용하지 않는다고 한다.

<br />

2. 나의 방법
<br />
    1. timeseries split으로 5fold를 나눈다.
<br />
    2. train set과 val셋의 기간 차이는 77일로 둔다. => 77일이 fold마가 행의 수가 다르기 때문에 잘 나눠야한다.
<br />
    3. 하이퍼파라미터 튜닝 진행
<br />
    4. 최적의 파라미터로 훈련된 모델의 평균 AUC값과 표준편차를 확인
    
어차피 Test셋을 정확히 반영하는 데이터셋을 만들 수는 없으므로 Test셋을 만들지 않는다.

3. ROC curve

시간이 있다면 5개 fold에 대해서 모두 그리자 => R 사용

In [40]:
# CV Class
class BigconTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
        
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y = None, groups = None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)
        
        # fold마다 margin을 달리 잡는다.
        margin = [77, 77, 77, 77, 77]
        start = 0
        stop = 0 
        for i in range(self.n_splits):
            stop = (i + 1) *  k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start:mid], indices[mid + margin[i]: stop]

In [41]:
import numpy as np
X = np.arange(10000)
y = np.arange(10000)
btscv = BigconTimeSeriesSplit(n_splits=5)
for train_index, test_index in btscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [   0    1    2 ... 1597 1598 1599] TEST: [1677 1678 1679 1680 1681 1682 1683 1684 1685 1686 1687 1688 1689 1690
 1691 1692 1693 1694 1695 1696 1697 1698 1699 1700 1701 1702 1703 1704
 1705 1706 1707 1708 1709 1710 1711 1712 1713 1714 1715 1716 1717 1718
 1719 1720 1721 1722 1723 1724 1725 1726 1727 1728 1729 1730 1731 1732
 1733 1734 1735 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746
 1747 1748 1749 1750 1751 1752 1753 1754 1755 1756 1757 1758 1759 1760
 1761 1762 1763 1764 1765 1766 1767 1768 1769 1770 1771 1772 1773 1774
 1775 1776 1777 1778 1779 1780 1781 1782 1783 1784 1785 1786 1787 1788
 1789 1790 1791 1792 1793 1794 1795 1796 1797 1798 1799 1800 1801 1802
 1803 1804 1805 1806 1807 1808 1809 1810 1811 1812 1813 1814 1815 1816
 1817 1818 1819 1820 1821 1822 1823 1824 1825 1826 1827 1828 1829 1830
 1831 1832 1833 1834 1835 1836 1837 1838 1839 1840 1841 1842 1843 1844
 1845 1846 1847 1848 1849 1850 1851 1852 1853 1854 1855 1856 1857 1858
 1859 1860 1861 1862 1863 18

## 예전 모델들

1. Logistic Regression
2. Random Forest 
3. Support Vector Machine
4. Linear Discriminant Analysis
5. Quadratic Discriminant Analysis
6. KNNClassifier
7. RidgeClassifier

<br />
추가로, 필요하다면 선형모델들에는 GAM을 채용 

In [ ]:
rom sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsRestClassifier

from sklearn.impute import SimpleImputer

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score , ShuffleSplit
from sklearn import metrics
from sklearn.metrics import roc_curve , roc_auc_score , f1_score , classification_report
## plot
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets

##Model classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier 


def rocvis(true , prob , label ) :
    AUC = np.mean(true == np.round(prob.ravel()).astype(int))
    if type(true[0]) == str :
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        true = le.fit_transform(true)
    else :
        pass
    fpr, tpr, thresholds = roc_curve(true, prob)
    plt.plot(fpr, tpr, marker='.', label =  "AUC : {:.2f} , {}".format(AUC,label)   )
    
    
scale_features = [연속형 변수들]

tscv = TimeSeriesSplit(n_splits=5)
scores = ['AUC', 'accuracy']   

import warnings
warnings.filterwarnings("ignore")
from pycm import * 

cv = TimeSeriesSplit(n_splits=5 , test_size=0.3, random_state=8282)

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('clf', LogisticRegression(random_state=8282))])

pipe_rf = Pipeline([('scl', StandardScaler()),
                    ('clf', RandomForestClassifier(random_state=8282))])

pipe_svm = Pipeline([('scl', StandardScaler()),
                     ('clf', SVC(random_state=8282, probability = True))])

pipe_lda = Pipeline([('scl', StandardScaler()),
                     ('clf', LDA(n_components = 2))])

pipe_qda = Pipeline([('scl', StandardScaler()),
                     ('clf', QDA())])

pipe_knn = Pipeline([('scl', StandardScaler()),
                     ('clf', KNN())])

pipe_ridge = Pipeline([('scl', StandardScaler()),
                     ('clf', RidgeClassifier())])


aram_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
depth_range = [7, 8, 9, 10]
min_samples_split_range = [0.5, 0.7 , 0.9]
param_range = [0.5, 0.1]
param_range_fl = np.logspace(0,-5,5)

grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
                   'clf__C': param_range_fl,
                   'clf__solver': ['liblinear']}] 

grid_params_rf = [{
                   'clf__min_samples_leaf': param_range,
                   'clf__max_depth': depth_range,
                   'clf__min_samples_split': min_samples_split_range }]

grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
                    'clf__C': param_range}]

grid_params_lda = [{"clf__tol" : param_range_fl , 
                    "clf__solver" : ["svd","lsqr","eigen"]} ]

grid_params_qda = [{"clf__tol" : param_range_fl} ]


grid_params_knn = [{"clf__n_neighbors" : [2,4,6] , 
                    "clf__weights" : ["uniform","distance"] , 
                    'clf__algorithm': ["ball_tree", "kd_tree", "brute"],
                   } ]

grid_params_ridge = [
    {"clf__solver" : ["svd","cholesky","lsqr","sparse_cg" , "sag","saga"] ,
     "clf__tol" : param_range_fl , 
     "clf__alpha" : param_range_fl , 
    }
]


pipe = [
    pipe_lr , pipe_rf , pipe_svm , 
    pipe_lda , pipe_qda , pipe_gbm , 
    pipe_knn , pipe_ridge
       ]
params = [
    grid_params_lr , grid_params_rf , grid_params_svm , 
    grid_params_lda , grid_params_qda, grid_params_gbm , 
    grid_params_knn , grid_params_ridge , 
]

jobs = 20

grid_dict = {0: 'Logistic Regression', 
             1: 'Random Forest', 
             2: 'Support Vector Machine' ,
             3: "Linear Discriminant Analysis",
             4: "Quadratic Discriminant Analysis",
             5: "KNNClassifier" , 
             6: "RidgeClassifier" , 
            }

model_prob = {}
model_result = {}
model_best_params = {}
model_confusion = {}
plt.style.use('ggplot')
fig , ax = plt.subplots(figsize= (20,10))
plt.plot([0, 1], [0, 1], linestyle='--')


for idx , (param , model) in enumerate(zip(params , pipe)) :
    search = GridSearchCV(model, param, iid=True , 
                          cv=cv , n_jobs=jobs , verbose=-1 )
    search.fit(X_train , y_train)
    y_pred = search.predict(X_test)
    try :
        y_prob = search.predict_proba(X_test)
    except Exception as e :
        pass
    rocvis(true = y_test , prob = y_prob[:,1] , label = grid_dict.get(idx) )
    model_result[grid_dict.get(idx)] = roc_auc_score(y_test, y_pred)  
    model_prob[grid_dict.get(idx)] = y_prob
    model_best_params[grid_dict.get(idx)] = search.best_params_
    model_confusion[grid_dict.get(idx)] = ConfusionMatrix(y_test, y_pred)
    
plt.legend(fontsize = 20 , loc='center', shadow=True )
plt.title("Models Roc Curve" , fontsize= 25)
plt.savefig("./Model_Result.png")
plt.show()

cp = Compare(model_confusion)

## LightGBM, CatBoost

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

fit_params={"early_stopping_rounds":300, 
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(X_test, y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': False,
            'categorical_feature': 'auto'}

def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate  * np.power(.995, current_iter)
    return max(lr, min_learning_rate)

fit_params['callbacks'] = [lgb.reset_parameter(learning_rate=learning_rate_power_0997)]

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(12, 20), 
             'min_child_samples': sp_randint(40, 100), 
             #'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.75, scale=0.25), 
             'colsample_bytree': sp_uniform(loc=0.8, scale=0.15)#,
             #'reg_alpha': [0, 1e-3, 1e-1, 1, 10, 50, 100],
             #'reg_lambda': [0, 1e-3, 1e-1, 1, 10, 50, 100]
            }

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.05, objective='multiclass',
                         random_state=314, silent=True, metric='None', 
                         n_jobs= -1, n_estimators=500, class_weight='balanced')

tscv = TimeSeriesSplit(n_splits = 5)

gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring=['f1_macro', 'accuracy'],
    cv= tscv,
    refit='f1_macro',
    random_state=314, 
    verbose=True)

_ = gs.fit(X_train, y_train, **fit_params)

print("PERFORMANCE IMPROVES FROM TOP TO BOTTOM")
print("Valid+-Std     Train  :   Parameters")
for i in np.argsort(gs.cv_results_['mean_test_f1_macro'])[-5:]:
    print('{1:.3f}+-{3:.3f}     {2:.3f}   :  {0}'.format(gs.cv_results_['params'][i], 
                                    gs.cv_results_['mean_test_f1_macro'][i], 
                                    gs.cv_results_['mean_train_f1_macro'][i],
                                    gs.cv_results_['std_test_f1_macro'][i]))

opt_parameters = gs.best_params_

means_accuracy = gs.cv_results_['mean_test_accuracy']
stds_accuracy = gs.cv_results_['std_test_accuracy']
for mean, std, params in zip(means_accuracy, stds_accuracy, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

# Fit the final model with learning rate decay
clf_final = lgb.LGBMClassifier(**clf.get_params())
clf_final.set_params(**opt_parameters)

def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate  * np.power(.997, current_iter)
    return max(lr, min_learning_rate)

#Train the final model with learning rate decay
fit_params['verbose'] = 200
_ = clf_final.fit(X_train, y_train, **fit_params)#, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_power_0997)])

## RNN